winget install 'Microsoft .NET SDK 7.0'  #Needed. Doesn't work on .Net 8
### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/2/2023 9:00:01 PM
Boot                  Sat 12.2.23 10:36
CpuUsage%             76
RamUsage%             {51}
C:%Free               {55}



This shows which processes are consuming the most cpu, memory and network and have the most number of handles and threads currently on the system.

In [7]:
$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads
$nwAggregated = @{}

for ($j = 0; $j -lt 3; $j++) {Start-Sleep -Seconds 3
    Get-Process | ForEach-Object {$proc = $objArr | Where-Object ID -EQ $_.Id ; $proc.CPU += $_.CPU ; $proc.WS += $_.WS }

    $nw = (Get-Counter '\Process(*)\IO Read Operations/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"}) |% {$nwAggregated[$_.InstanceName] += $_.CookedValue}
}

$Network = $nwAggregated.GetEnumerator() | Sort-Object Value -Descending | Select-Object -First 10

$harray = for ($j = 0; $j -lt 10; $j++) { $sortedObjArr = $objArr | Sort-Object CPU, WS, HandleCount, Threads -Descending

    [PSCustomObject][ordered]@{
        'CPU'          = $sortedObjArr[$j].Process
        'WS'           = $sortedObjArr[$j].Process
        'Handles'      = $sortedObjArr[$j].Process
        'Threads'      = $sortedObjArr[$j].Process
        'Network'      = $Network[$j].Name
    }
}
$harray | Format-Table -AutoSize


CPU                    WS                     Handles                Threads                Network
---                    --                     -------                -------                -------
msedge►17220           msedge►17220           msedge►17220           msedge►17220           code -…
powershell_ise►10100   powershell_ise►10100   powershell_ise►10100   powershell_ise►10100   audiodg
Grammarly.Desktop►7320 Grammarly.Desktop►7320 Grammarly.Desktop►7320 Grammarly.Desktop►7320 dwm
Code - Insiders►2860   Code - Insiders►2860   Code - Insiders►2860   Code - Insiders►2860   window…
msedge►9840            msedge►9840            msedge►9840            msedge►9840            msedge
Code - Insiders►12740  Code - Insiders►12740  Code - Insiders►12740  Code - Insiders►12740  svchost
msedge►10628           msedge►10628           msedge►10628           msedge►10628           wudfho…
explorer►8092          explorer►8092          explorer►8092          explorer►8092          dllhost
OneD